In [1]:
import sqlite3, json, os

# Paths
DB_PATH  = 'database/book_reviews.db'
if os.path.exists(DB_PATH):
    os.remove(DB_PATH)
SCHEMA   = './db_schema.sql'
PHRASES  = 'data/processed/phrases.json'
os.makedirs(os.path.dirname(DB_PATH), exist_ok=True)

In [2]:
# Connect & create tables
conn = sqlite3.connect(DB_PATH)
cur  = conn.cursor()
with open(SCHEMA,'r',encoding='utf-8') as f:
    cur.executescript(f.read())


In [3]:
# Load your JSON data
with open(PHRASES,'r',encoding='utf-8') as f:
    data = json.load(f)

In [4]:
# Insert into tables
for entry in data:
    asin    = entry.get('asin','')
    title   = entry.get('title','')
    genre   = entry.get('genre','')
    price   = entry.get('price','')
    if asin == "0006498442":
        print(genre)
    # Books table
    cur.execute(
        "INSERT OR IGNORE INTO books (asin,title,price,genre) VALUES (?,?,?,?)",
        (asin, title, price, genre)
    )

    # Reviews table
    cur.execute(
        "INSERT INTO reviews (asin,rating,review_text,summary) VALUES (?,?,?,?)",
        (asin, entry.get('rating'), entry.get('text',''), entry.get('summary',''))
    )
    review_id = cur.lastrowid

    # Sentiments table
    for p in entry.get('phrases', []):
        cur.execute(
            "INSERT INTO sentiments (review_id,phrase,polarity) VALUES (?,?,?)",
            (review_id, p['text'], p['polarity'])
        )

# Commit and close
conn.commit()
conn.close()
print("Data insertion complete")


Data insertion complete


In [5]:
# Verify & Query Counts
import sqlite3

conn = sqlite3.connect('database/book_reviews.db')
for table in ('books','reviews','sentiments'):
    cnt = conn.execute(f"SELECT COUNT(*) FROM {table}").fetchone()[0]
    print(f"{table}: {cnt} rows")
conn.close()


books: 28609 rows
reviews: 537431 rows
sentiments: 1709033 rows
